In [1]:
from VariationalAutoDecoder_Beta import VariationalAutoDecoder as VAD_Beta
from VAD_Trainer import VAD_Trainer
import utils
from evaluate import evaluate_model
import torch
import torch.optim as optim
import torch.nn as nn
import csv
import time
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Create DataLoaders

In [2]:
train_ds, train_dl, test_ds, test_dl = utils.create_dataloaders(data_path="dataset" ,batch_size=64)

## Train Auto Decoder

In [ ]:
import torch.nn as nn

architectures = [
    {
        "mu_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        )
    },

    {
        "mu_net": nn.Sequential(
            nn.Linear(128, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )
    },

    {
        "mu_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )
    }
]



# latent_dims = [dim for dim in [64, 32, 128, 16, 10] for _ in range(5)]
VADs = [VAD(mu_layers=arch['mu_net'], var_layers=arch['log_var_net'], device=device) for arch in architectures]# for _ in range(5)]
# learning_rates = [lr for lr in [0.001, 0.0005, 0.0001, 0.002, 0.005] for _ in range(5)]
trainers = [VAD_Trainer(var_decoder=VADs[i], dataloader=train_dl, latent_dim=128, device=device, lr=1e-2) for i in range(len(VADs))]

In [ ]:
# Initialize the results list to hold all the data
num_test_samples = len(test_dl.dataset)

# Create latent parameters and optimizers for each trainer
temp_latents = torch.randn(10, 128).to(device)
latents_list = [torch.nn.Parameter(torch.stack([temp_latents[label,:] for label in train_dl.dataset.y])).to(device) for i in range(len(VADs))]
optimizers = [optim.Adam([latents], lr=1e-3) for latents in latents_list]

# Save results to a CSV file
csv_file_path = 'results_VAD_temp.csv'

# Write header to the CSV file first
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['Index'] + [f'Epoch {i+1} Loss' for i in range(500)] + ['Final Test Loss']
    writer.writerow(header)

# Main training and evaluation loop
for index, trainer in enumerate(trainers):
    start_time = time.time()  # Record the start time
    train_loss = trainer.train(num_epochs=500)  # Train the model
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Trainer {index} has finished training in {elapsed_time:.2f} seconds.")

    start_time = time.time()  # Record the start time
    test_loss = evaluate_model(model=VADs[index], test_dl=test_dl, opt=optimizers[index], latents=latents_list[index], epochs=500, device=device) 
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"AD {index} has finished test evaluation in {elapsed_time:.2f} seconds.")

    # Prepare the row to be saved
    row = [index] + train_loss + [test_loss]

    # Append results to the CSV file after each iteration
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

print(f"Results saved to {csv_file_path}.")

In [ ]:
for i in range(len(trainers)):
    utils.plot_tsne(train_ds, trainers[i].latents, f"tsne_kl_{i}")

## Fine Tune

In [ ]:
arch = {
        "mu_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        )
    }

learning_rates = [0.001, 0.0001, 0.005]
VADs_ft = [VAD(mu_layers=arch['mu_net'], var_layers=arch['log_var_net'], device=device) for _ in range(len(learning_rates))]# for _ in range(5)]
trainers_ft = [VAD_Trainer(var_decoder=VADs_ft[i], dataloader=train_dl, latent_dim=128, device=device, lr=learning_rates[i]) for i in range(len(VADs_ft))]

In [ ]:
# Initialize the results list to hold all the data
num_test_samples = len(test_dl.dataset)

# Create latent parameters and optimizers for each trainer
temp_latents_ft = torch.randn(10, 128).to(device)
latents_list_ft = [torch.nn.Parameter(torch.stack([temp_latents_ft[label,:] for label in train_dl.dataset.y])).to(device) for i in range(len(VADs_ft))]
optimizers = [optim.Adam([latents], lr=1e-3) for latents in latents_list_ft]

# Save results to a CSV file
csv_file_path = 'results_VAD_ft.csv'

# Write header to the CSV file first
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['Index'] + [f'Epoch {i+1} Loss' for i in range(500)] + ['Final Test Loss']
    writer.writerow(header)

# Main training and evaluation loop
for index, trainer in enumerate(trainers_ft):
    start_time = time.time()  # Record the start time
    train_loss = trainer.train(num_epochs=500)  # Train the model
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Trainer {index} has finished training in {elapsed_time:.2f} seconds.")

    start_time = time.time()  # Record the start time
    test_loss = evaluate_model(model=VADs_ft[index], test_dl=test_dl, opt=optimizers[index], latents=latents_list_ft[index], epochs=500, device=device) 
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"AD {index} has finished test evaluation in {elapsed_time:.2f} seconds.")

    # Prepare the row to be saved
    row = [index] + train_loss + [test_loss]

    # Append results to the CSV file after each iteration
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

print(f"Results saved to {csv_file_path}.")

In [ ]:
for i in range(len(trainers_ft)):
    utils.plot_tsne(train_ds, trainers_ft[i].latents, f"tsne_kl_ft_{i}")

## Best model

In [3]:
arch = {
        "mu_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        )
    }

VAD_best = VAD_Beta(mu_layers=arch['mu_net'], var_layers=arch['log_var_net'], device=device)
trainer_best = VAD_Trainer(var_decoder=VAD_best, dataloader=train_dl, latent_dim=128, device=device, lr=0.001)
trainer_best.train(num_epochs=1000)

Epoch [1/1000], Loss: 0.4978
Epoch [2/1000], Loss: 0.4725
Epoch [3/1000], Loss: 0.4522
Epoch [4/1000], Loss: 0.4314
Epoch [5/1000], Loss: 0.4100
Epoch [6/1000], Loss: 0.3879
Epoch [7/1000], Loss: 0.3662
Epoch [8/1000], Loss: 0.3450
Epoch [9/1000], Loss: 0.3245
Epoch [10/1000], Loss: 0.3072
Epoch [11/1000], Loss: 0.2919
Epoch [12/1000], Loss: 0.2776
Epoch [13/1000], Loss: 0.2651
Epoch [14/1000], Loss: 0.2579
Epoch [15/1000], Loss: 0.2544
Epoch [16/1000], Loss: 0.2565
Epoch [17/1000], Loss: 0.2574
Epoch [18/1000], Loss: 0.2505
Epoch [19/1000], Loss: 0.2469
Epoch [20/1000], Loss: 0.2436
Epoch [21/1000], Loss: 0.2407
Epoch [22/1000], Loss: 0.2368
Epoch [23/1000], Loss: 0.2328
Epoch [24/1000], Loss: 0.2311
Epoch [25/1000], Loss: 0.2308
Epoch [26/1000], Loss: 0.2305
Epoch [27/1000], Loss: 0.2299
Epoch [28/1000], Loss: 0.2272
Epoch [29/1000], Loss: 0.2246
Epoch [30/1000], Loss: 0.2223
Epoch [31/1000], Loss: 0.2210
Epoch [32/1000], Loss: 0.2190
Epoch [33/1000], Loss: 0.2174
Epoch [34/1000], Lo

[0.4977957271039486,
 0.4725069981068373,
 0.452162591740489,
 0.4314137063920498,
 0.4099830389022827,
 0.3879004120826721,
 0.3662266358733177,
 0.34498695097863674,
 0.32453292049467564,
 0.3071757238358259,
 0.2918539959937334,
 0.27758412156254053,
 0.2650839611887932,
 0.2578728049993515,
 0.2543589826673269,
 0.2565208626911044,
 0.25738430209457874,
 0.2505321642383933,
 0.24687490053474903,
 0.24360256362706423,
 0.24072967655956745,
 0.2368289725854993,
 0.23284560814499855,
 0.23114890605211258,
 0.23084702715277672,
 0.2305209757760167,
 0.22993244975805283,
 0.22722960636019707,
 0.2246315125375986,
 0.22226075641810894,
 0.22102274931967258,
 0.21903486270457506,
 0.21737282909452915,
 0.21608473360538483,
 0.21539924945682287,
 0.2144812848418951,
 0.21376310102641582,
 0.21323053538799286,
 0.21219651773571968,
 0.21146018616855145,
 0.21103897131979465,
 0.20956595614552498,
 0.21038893423974514,
 0.212104681879282,
 0.21032448019832373,
 0.20856119133532047,
 0.207411

In [6]:
utils.plot_tsne(train_ds, trainer_best.latents, f"tsne_Beta")

<Figure size 800x600 with 0 Axes>

## Sample specific vectors

In [4]:
num_test_samples = len(test_dl.dataset)
temp_latents_best = torch.randn(10, 128).to(device)
latents_best = torch.nn.Parameter(torch.stack([temp_latents_best[label,:] for label in test_dl.dataset.y])).to(device)
opt = optim.Adam([latents_best], lr=1e-3)

In [5]:
test_loss = evaluate_model(model=VAD_best, test_dl=test_dl, opt=opt, latents=latents_best, epochs=1000, device=device)
print(f"AD has finished test evaluation with a test loss of {test_loss}.")

AD has finished test evaluation with a test loss of 0.23139685671776533.


In [9]:
utils.plot_tsne(test_ds, latents_best, f"tsne_test_beta")

<Figure size 800x600 with 0 Axes>

In [10]:
# Randomly sample 5 indices from the test dataset
random.seed(6)
sampled_indices = random.sample(range(len(latents_best)), 5)

# Extract the corresponding vectors (input data) and their labels
sampled_latents = [latents_best[i] for i in sampled_indices]  # Only selecting input data, not labels

# Convert to a single tensor (optional)
sampled_latents_tensor = torch.stack(sampled_latents)
random_latents_tensor = torch.randn_like(sampled_latents_tensor)

sampled_test_images = VAD_best(sampled_latents_tensor).view(-1, 1, 28, 28)
random_test_images = VAD_best(random_latents_tensor).view(-1, 1, 28, 28)

utils.save_images(sampled_test_images, "sampled_test_images_VAD_beta.png")
utils.save_images(random_test_images, "random_test_images_VAD_beta.png")

## Interpolation

In [8]:
import numpy as np
random.seed(56)
sampled_indices = random.sample(range(len(latents_best)), 2)
sampled_latents = [latents_best[i] for i in sampled_indices]
weights = np.linspace(0, 1, 7)
interpolated_latents = [w * sampled_latents[0] + (1 - w) * sampled_latents[1] for w in weights]
interpolated_latents_tensor = torch.stack(interpolated_latents)
interpolated_images = VAD_best(interpolated_latents_tensor).view(-1, 1, 28, 28)
utils.save_images(interpolated_images, "interpolated_images_beta.png")